In [ ]:
!pip install faker

# NER 모델 설정 및 로드


### 필요한 파이썬 패키지 설치


일래스틱은 허깅 페이스에서 모델을 다운로드하고 일래스틱서치로 로드하기 위해 [eland 파이썬 라이브러리](https://github.com/elastic/eland)를 사용합니다.

In [ ]:
!pip install eland elasticsearch transformers sentence_transformers torch==1.11

In [ ]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

### 일래스틱서치 인증 설정
[일래스틱 클라우드 ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html)와 [클러스터 API 키](https://www.elastic.co/guide/en/kibana/current/api-keys.html)를 사용한 인증 접근 방식을 권장합니다.

자격 증명을 하기 위한 어떠한 방법을 사용해도 상관없습니다. 아래 예제에서는 깃 저장소에 자격 증명을 저장하지 않고 보안을 유지하기 위해 getpass 모듈을 사용하여 자격 증명을 입력합니다.

In [ ]:
import getpass

In [ ]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_api_key = getpass.getpass('Enter cluster API key:  ')

### 일래스틱 클라우드 접속

In [ ]:
es = Elasticsearch(cloud_id=es_cloud_id,
                   api_key=es_api_key
                   )
es.info() # 클러스터 정보 확인

## 허깅 페이스 모델 허브에서 모델을 다운로드해 일래스틱서치로 가져오기

일래스틱 스택의 머신러닝 기능은 표준 BERT 모델 인터페이스를 준수하고 WordPiece 토큰화 알고리즘을 사용하는 변환 모델을 지원합니다.

현재 지원되는 아키텍처 목록은 다음에서 볼 수 있습니다. [일래스틱 NLP 모델 지원 목록](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-model-ref.html)



### HF 복사 링크를 사용하여 허깅 페이스에서 NER 모델 다운로드
예제에서는 [sentence-transformers/msmarco-MiniLM-L-12-v3](https://huggingface.co/sentence-transformers/msmarco-MiniLM-L-12-v3) 모델을 사용할 것입니다.


### 모델 다운로드
여기서는 Hugging Face의 모델 ID를 지정합니다. 이 ID를 얻는 가장 쉬운 방법은 모델 페이지에서 이름 옆에 있는 모델 이름 아이콘을 클릭하여 복사하는 것입니다.

`TransformerModel`을 호출할 때 HF 모델 ID와 작업 유형을 지정합니다. auto를 지정하여 eland가 모델 구성 정보에서 자동으로 유형을 결정하도록 시도할 수 있습니다. 그러나 이것이 항상 가능한 것은 아니며 특정 task_type 값의 목록은 [지원되는 task_type 코드](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)에서 볼 수 있습니다.

_"Some weights of the model checkpoint"에 대한 경고는 무시할 수 있습니다._

In [ ]:
hf_model_id='dslim/bert-base-NER'
tm = TransformerModel(model_id=hf_model_id, task_type="ner")

### 모델 ID 설정 및 확인
이름이 일래스틱서치와 호환되도록 하기 위해서 '/'는 '__'로 대체됩니다.



In [ ]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

## 일래스틱서치가 사용하는 TorchScrpt로 모델 내보내기

In [ ]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

## 일래스틱서치에 모델을 로드
일래스틱서치에 모델이 없어야 합니다.

In [ ]:
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

## 모델 시작

### 모델에 대한 정보 확인
필수는 아니지만 모델 개요를 얻는 데 유용합니다.

In [ ]:
# 일래스틱서치에 있는 목록
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

### 모델 배포
ML 노드에 모델이 로드되고 NLP 작업에 사용할 수 있도록 프로세스가 시작됩니다.

In [ ]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

### 모델이 문제없이 시작되었는지 확인

In [ ]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

# 파이프라인 생성

In [ ]:
description= "PII redacting ingest pipeline"

processors= [
    {
      "set": {
        "field": "redacted",
        "value": "{{{message}}}"
      }
    },
    {
      "inference": {
        "model_id": es_model_id,
        "field_map": {
          "message": "text_field"
        }
      }
    },
    {
      "script": {
        "lang": "painless",
        "source": """String msg = ctx['message'];
      for (item in ctx['ml']['inference']['entities']) {
        msg = msg.replace(item['entity'], '<' + item['class_name'] + '>')
      }
      ctx['redacted']=msg""",
        "if": "return ctx['ml']['inference']['entities'].isEmpty() == false",
        "tag": "ner_redact",
        "description": "Redact NER entities"
      }
    },
    {
      "redact": {
        "field": "redacted",
        "patterns": [
          "%{PHONE:PHONE}",
          "%{SSN:SSN}"
        ],
        "pattern_definitions": {
          "SSN": """\d{6}-?\d{7}""",
          "PHONE": """\d{3}-?\d{3}-?\d{4}"""
        }
      }
    },
    {
      "remove": {
        "field": [
          "message",
          "ml"
        ]
      }
    }
  ]

on_failure= [
    {
      "set": {
        "field": "failure",
        "value": "pii_script-redact"
      }
    }
  ]



response = es.ingest.put_pipeline(id="pii_redaction_pipeline_book",
                                  description=description,
                                  processors=processors,
                                  on_failure=on_failure
)


# Print the response
print(response)

# 인덱스 템플릿 설정
`pii_data*` 패턴으로 생성된 모든 인덱스를 찾습니다.
필드에 대한 매핑을 작성합니다.
새 데이터가 기본적으로 `pii_redaction_pipeline`을 사용하도록 구성합니다.


In [ ]:
index_patterns =[
    "pii_data*"
  ]
order = 1
settings = {
      "number_of_shards": 1,
      "number_of_replicas": 1,
      "index.default_pipeline": "pii_redaction_pipeline_book"
    }
mappings = {
      "properties": {
        "message": {
          "type": "text"
        },
        "status": {
          "type": "keyword"
        },
        "redacted": {
          "type": "text"
        }
      }
    }


# 인덱스 템플릿 생성
response = es.indices.put_template(name="pii_book_template",
                                   index_patterns=index_patterns,
                                   order=order,
                                   settings=settings,
                                   mappings=mappings
                                   )

# 응답값 출력
print(response)


# 가짜 PII 데이터 생성

In [ ]:
from faker import Faker
import json
from pprint import pprint

# Faker 클래스 인스턴스 생성
fake = Faker('ko_kr')

# 가짜 PII 생성 함수 정의
def generate_fake_pii(num_records):

  fake_data = []

  for x in range(num_records):
    # 가짜 PII 생성
    fn = fake.first_name()
    ln = fake.last_name()
    pn = fake.phone_number()
    sn = fake.ssn()
    ai = fake.random_element(elements=('활성화', '비활성화'))

    call_log = {
        'message' : f'{ln}{fn}의 전화번호는 {pn} 이고 주민등록번호는 {sn} 입니다.',
        'status' : ai
        }

    fake_data.append(call_log)
  return fake_data

# N 명의 가짜 PII 정보 생성
num_records = 10 # 가짜 PII 정보를 생성할 숫자 설정
fake_pii_data = generate_fake_pii(num_records)

pprint(fake_pii_data)

# 일래스틱서치에 가짜 데이터 수집
데이터를 pii_data 인덱스로 저장하면 pii 제거 파이프라인을 통해 저장됩니다.

In [ ]:
from elasticsearch import Elasticsearch, helpers

# 대량 인덱서
# 일래스틱서치 문서 배열을 만드는 함수 정의
def generate_documents_array(fake_data, index_name):

  # 문서를 저장할 빈 배열 만들기
  documents = []

  # 가짜 데이터 목록을 문서 형식에 맞게 변환
  for call in fake_data:
    # _index와 _source 키를 사용해 문서 만들기
    document = {
      '_index': index_name,
      '_source': call
    }

    # 문서 배열에 문서 추가
    documents.append(document)

  return documents


# 일래스틱서치 문서 배열 만들기
index_name = 'pii_data-book'  # 인덱스 이름 설정
documents_array = generate_documents_array(fake_pii_data, index_name)

# 대량 색인 요청 본문을 개행 구분자를 사용해 단일 문자로 출력
print("Bulk request: ")
print(documents_array)

try:
	response = helpers.bulk(es, documents_array)
	print ("\nRESPONSE:", response)
except Exception as e:
	print("\nERROR:", e)

In [ ]:
query = {"match_all":{}}

response = es.search(
    index=index_name,
    query=query
    )

pprint(response['hits']['hits'])